In [1]:
import pandas as pd
import numpy as np
import psycopg2
from glob import glob
from sqlalchemy import create_engine
from urllib.parse import quote
import tqdm
import os

* #### Reading the occupation file with the verbatim count

In [2]:
df_occupation_count = pd.read_csv(r"C:\Users\FIU\Desktop\EFT_MCA\occupation\occupation_list_15_11_22.csv", delimiter='\t')

In [3]:
df_occupation_count

,occupation,Count
0,NaN,10750337.0
1,BUSINESS,2746051.0
2,OTHERS,2356190.0
3,BANKING,1251374.0
4,OTHER,611961.0
...,...,...
13042,HOTEL K ROASE,1.0
13043,CHEMICAL TESTING AND CONSULTANCY,1.0
13044,BUSINESS - SELF EMPLOYED,1.0
13045,OBSOLETE IT RELATED,1.0


In [4]:
df_occupation_amount = pd.read_csv(r"C:\Users\FIU\Desktop\EFT_MCA\occupation\occupation_all_inward_outward_15_11_22_final.csv", delimiter='\t')

In [5]:
df_occupation_amount

,occupation,outward_amount,inward_amount,outward_count,inward_count
0,PROPRIETOR OF CAREER ZONE CONSULTING,1.597425e+06,NaN,3.0,NaN
1,PARTNER IN PROFESSIO,3.714535e+07,NaN,7.0,NaN
2,V L,1.566072e+07,NaN,4.0,NaN
3,TELEVISION PROGRAMMING AND BROADCASTING ACTIVI...,2.344583e+10,244589661.0,533.0,68.0
4,BUELT,3.483976e+07,3674047.0,30.0,2.0
...,...,...,...,...,...
13042,PARTSANDACCESSORIESFORMETALWORKINGMACHIN,2.255088e+08,229133959.0,32.0,22.0
13043,"CREATIVE, ARTS AND E",NaN,16157912.0,NaN,15.0
13044,MANUFACTURE OF FRUIT AND VEGETABLE PRESERVES,NaN,6784777.0,NaN,4.0
13045,REGIONAL MANAGER NORTH AT 3 M INDIA LTD,1.815083e+06,NaN,3.0,NaN


In [6]:
# reading fuzzy_wuzzy_merged_file

df_fuzzy_wuzzy = pd.read_csv(r"C:\Users\FIU\Desktop\EFT_MCA\occupation\merged_fuzzy_wuzzy.csv")

In [7]:
df_fuzzy_wuzzy.drop("Unnamed: 0", axis = 1, inplace = True)

In [8]:
# reading substring_merged_file

df_sub_string = pd.read_csv(r"C:\Users\FIU\Desktop\EFT_MCA\occupation\merged_with_substring.csv")

In [9]:
# reading non_fuzzy_wuzzy_file

df_not_fuzzy = pd.read_csv(r"C:\Users\FIU\Desktop\EFT_MCA\occupation\mapped_occupations_without_fuzzy.csv")

In [10]:
df_not_fuzzy

,OccupationDesc,Occupation,correct Form
0,ACADEMIC,ACADEMIC,Student
1,ACADEMIC,ACADAMIC,NaN
2,ACADEMIC,ACADEMIC,NaN
3,ACADEMIC,ACADEMICIAN,NaN
4,ACCOUNTANT,ACCONT,salaried
...,...,...,...
4526,FACTORY,SUGAR FACTORIES,NaN
4527,POLICE,POLIC,NaN
4528,POLICE,POLICE,NaN
4529,POLICE,POLICE FORCES,NaN


In [11]:
df_sub_string.drop("Unnamed: 0", axis = 1, inplace = True)

In [12]:
df_sub_string.columns = ['OccupationDesc', 'Occupation']

In [13]:
df_sub_string

,OccupationDesc,Occupation
0,ACADEMIC,ACADEMIC
1,ACADEMIC,ACADEMIC
2,ACADEMIC,ACADEMICIAN
3,ADMINISTRATION,ADMINISTRATION
4,ADMINISTRATION,ADMINISTRATION
...,...,...
310,TRADER,TRADE TRADERS
311,TRADER,TRADER
312,TRADER,TRADERS
313,VICE PRESIDENT,VICE PRESIDENT


In [14]:
df_fuzzy_wuzzy.columns = ['OccupationDesc', 'Occupation']

In [15]:
df_fuzzy_wuzzy

,OccupationDesc,Occupation
0,ADMINISTRATION,ADMINISTRADOR
1,ADMINISTRATION,ADMINISTRATION
2,ADMINISTRATION,ADMINISTRATIVA
3,ADMINISTRATION,ADMINISTRATIVO
4,ADMINISTRATION,ADMINISTRATOR
...,...,...
370,WAITER,WATER
371,WAITER,WRITER
372,WEALTH TRUSTEES,WEALTH TRUST
373,WEALTH TRUSTEES,WEALTH TRUSTEES


In [16]:
df_final_mapping = pd.concat([df_fuzzy_wuzzy, df_sub_string, df_not_fuzzy]).sort_values(by = 'OccupationDesc').reset_index()

In [17]:
df_final_mapping.drop('index', axis = 1, inplace = True)

In [18]:
matches=[]
for i,j in zip(df_final_mapping['OccupationDesc'], df_final_mapping['Occupation']):
    if i == j:
        matches.append('Exact Match')
    else:
        matches.append('Not Exact')

In [19]:
df_final_mapping['matches_type'] = matches

In [20]:
df_final_mapping['matches_type'].value_counts()

Not Exact      4883
Exact Match     338
Name: matches_type, dtype: int64

In [21]:
df_grouped_mapping = df_final_mapping.groupby('OccupationDesc').aggregate({'Occupation': lambda x: ','.join(set(x))}).reset_index()

In [22]:
# pd.DataFrame.from_dict(dict(zip(df_grouped_mapping['OccupationDesc'], df_grouped_mapping['Occupation'])), orient='index').to_excel('dictionary_mapping.xlsx')

* **Mapping the identified occupation**

In [23]:
mapping_df = pd.read_excel(r"C:\Users\FIU\Desktop\EFT_MCA\occupation\dictionary_mapping.xlsx")

In [24]:
mapping_df.columns = ['Occupation', 'Occupation_mapped']

In [25]:
mapping_df

,Occupation,Occupation_mapped
0,ACADEMIC,"ACADAMIC,ACADEMIC,ACADEMICIAN"
1,ACCOUNTANT,"ACOUNTTANT,ACTUARY,STAFF ACCO,ACOUNT,ACCOUNTAN..."
2,ACTOR,"BOLLYWOOD ACTOR,ACTOR ACTRESS,FILM ACTOR,ACTRE..."
3,ADMINISTRATION,"ADMINISTRATIVA,ADMINISTRATOR,ADMINISTRADORA,AD..."
4,ADVOCATE,"ADVOCATE,LOWYER,LAWYER,AVVOCATO,ADVOC,LWYRS,LA..."
...,...,...
204,WAITER,"WRITER,WATER,WAITRESS,WAITER"
205,WATER CARRIER,"WATER PURIFIER,WATER SUPPLY,WATER,WATER TRANSP..."
206,WEALTH TRUSTEES,"WEALTH TRUSTEES,WEALTH TRUST"
207,WHOLESALER,"WHOLESALERS,WHOLESALER,WHOLE,WHOLESALE,WHOLESA..."


In [26]:
mapping_df = mapping_df.set_index('Occupation')\
.Occupation_mapped.str.split(',', expand = True)\
.stack()\
.reset_index('Occupation')\
.rename(columns = {0: 'Occupation_to_be_mapped'})\
.sort_values('Occupation_to_be_mapped')[['Occupation', 'Occupation_to_be_mapped']]\
.reset_index()

In [27]:
mapping_df.drop('index',axis = 1 ,inplace = True)

In [28]:
mapping_df.columns = ['Occupation_mapped_as','Occupation_to_be_mapped']

In [29]:
mapping_df['Occupation_mapped_as'] = mapping_df['Occupation_mapped_as'].str.upper()

In [30]:
df_occupation_count.columns = df_occupation_count.columns.str.upper()

In [31]:
df_occupation_amount.columns = df_occupation_amount.columns.str.upper()

In [32]:
df_occupation_amount['OCCUPATION']  = df_occupation_amount['OCCUPATION'].fillna('Not-available')

In [33]:
df_occupation_count['OCCUPATION'] = df_occupation_count['OCCUPATION'].fillna('Not-available')

In [34]:
df_occupation_amount.isna().sum()

OCCUPATION           0
OUTWARD_AMOUNT    2577
INWARD_AMOUNT     3646
OUTWARD_COUNT     2577
INWARD_COUNT      3646
dtype: int64

In [35]:
df_occupation_count.isna().sum()

OCCUPATION    0
COUNT         2
dtype: int64

In [36]:
df_occupation_count = df_occupation_count.groupby('OCCUPATION').aggregate({'COUNT':'sum'}).sort_values(by = 'COUNT', ascending=False).reset_index()

In [37]:
df_occupation_amount = df_occupation_amount.groupby('OCCUPATION').aggregate({'OUTWARD_AMOUNT':'sum','INWARD_AMOUNT':'sum', 'OUTWARD_COUNT':'sum', 'INWARD_COUNT':'sum'}).reset_index()

In [38]:
df_occupation_all = pd.merge(df_occupation_count, df_occupation_amount, on = 'OCCUPATION', how = 'inner')

In [39]:
mapping_dic = dict(zip(mapping_df['Occupation_to_be_mapped'], mapping_df['Occupation_mapped_as']))

In [42]:
pd.DataFrame.from_dict(mapping_dic, orient='index').reset_index().to_excel('mapping_dictionary.xlsx', index = False)

In [ ]:
mapping_dic.update({'AIRCRAFT MFG': 'AIR'})

In [ ]:
df_occupation = df_occupation_all.copy()

In [ ]:
df_occupation.OCCUPATION = df_occupation.OCCUPATION.replace(mapping_dic)

In [ ]:
df_occupation.columns

In [ ]:
df_occupation = df_occupation\
.groupby('OCCUPATION')\
.aggregate({'COUNT':'sum', 'OUTWARD_AMOUNT':'sum', 'INWARD_AMOUNT':'sum', 'OUTWARD_COUNT':'sum', 'INWARD_COUNT':'sum'})\
.sort_values(by = 'COUNT', ascending=False)

In [ ]:
df_occupation

In [ ]:
df_occupation_800 = df_occupation[:800]

In [ ]:
df_occupation_800_grouped = df_occupation_800.groupby('OCCUPATION').aggregate({'COUNT':'sum', 'OUTWARD_AMOUNT':'sum', 'INWARD_AMOUNT':'sum', 'OUTWARD_COUNT':'sum', 'INWARD_COUNT':'sum'}).sort_values(by = 'COUNT', ascending=False).reset_index()

In [ ]:
df_occupation_800_grouped

In [ ]:
df_occupation_500 = df_occupation_800_grouped[:500]

In [ ]:
df_occupation_500

In [ ]:
from rapidfuzz import fuzz
from rapidfuzz import process

In [ ]:
# empty lists for storing the
# matches later
mat1 = []
mat2 = []
mat3 = []

In [ ]:
list1 = df_occupation_500['OCCUPATION']
list2 = df_occupation_500['OCCUPATION']

In [ ]:
dframe_500 = df_occupation_500.copy()

In [ ]:
# iterating through list1 to extract 
# it's closest match from list2
for i in list1:
    mat1.append(process.extract(i, list2))
dframe_500['matches'] = mat1

In [ ]:
dframe_500

In [ ]:
p = []
l = []

In [ ]:
for j in dframe_500['matches']:
    for k in j:
        if k[1] >= 90:
            p.append(k[0])
            l.append(k[2])
    mat2.append((p))
    mat3.append(l)
    p = []
    l = []
# storing the resultant matches 
# back to dframe1
dframe_500['set_matches'] = mat2
dframe_500['set_matches_index'] = mat3
dframe_500

In [ ]:
#set_matches_index
dframe_500['set_matches_index_objct'] = dframe_500['set_matches_index'].apply(lambda x:sorted(x)).apply(lambda x: ''.join(str(x))).apply(lambda x: x.lstrip('[')).apply(lambda x: x.rstrip(']'))

In [ ]:
df_fuzzy = dframe_500.groupby('set_matches_index_objct').aggregate({'COUNT':'sum', 'OUTWARD_AMOUNT':'sum', 'INWARD_AMOUNT':'sum', 'OUTWARD_COUNT':'sum', 'INWARD_COUNT':'sum', 'set_matches': lambda x: ','.join(str(v) for v in x)}).sort_values(by = 'COUNT', ascending=False).reset_index()

In [ ]:
df_fuzzy

In [ ]:
df_fuzzy.to_excel('df_fuzzy_15_11_2022_V1.0.xlsx', index = False)